# Assignment 1: Dino Fun World

You, in your role as a burgeoning data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a couple questions about their park operations. In order to perform the requested analysis, they have provided you with a database containing information about one day of the park's operations.

### Provided Database

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkins', 'attractions', and 'sequences'. The information contained in each of these tables is listed below:

`checkins`:
    - Description: check-in data for all visitors for the day in the park. The data includes two types of check-ins, inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type
`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carussel, etc.
    - Fields: AttractionID, Name, Region, Category, type
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
The database is named 'dinofunworld.db' and is located in the 'readonly' directory of the Jupyter Notebook environment. It can be accessed at 'readonly/dinofunworld.db'.

### Questions to Answer

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.

Question 1: What is the most popular attraction to visit in the park?
Question 2: What ride (note that not all attractions are rides) has the longest visit time?
Question 3: Which Fast Food offering has the fewest visitors?
Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline.

#### Administrative Notes

This assignment will be graded by Coursera's grading system. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed.

In [460]:
import sqlite3
from time import time 
from datetime import datetime
import re 
from paretoset import paretoset

from datetime import datetime
def getsecs(time):
    t = time.split(':')
    
    hrs = int(t[0])
    mins = int(t[1])
    sec = (t[2])
    sec = int(sec[0:2])
    
    return sec + mins*60 + hrs*3600

In [461]:
getsecs('23:34:46@SD')

84886

In [462]:
# Graded Cell
# PartID:  NDnou
# Question 1: What is the most popular attraction to visit in the park?
# Notes: Your output should be the name of the attraction.

dino_file = 'dinofunworld.db'
connec = sqlite3.connect(dino_file)
c = connec.cursor()

c.execute(" SELECT attraction,count(attraction) from checkin GROUP BY attraction ")
dic = c.fetchall()
temp = 0
pos = 0
for i in dic:
    if i[1] >= temp:
        pos = i[0]
        temp = i[1]
#print(pos)
c.execute(" select name from attraction where attractionID = ?",(pos,))
print(c.fetchall()[0][0])



Atmosfear


In [463]:
# Graded Cell
# PartID: FXGHp
# Question 2: What ride (note that not all attractions are rides) has the longest average visit time?
# Notes: Your output should be the name of the ride.

#for row in c.execute("select attraction from (Select attraction,max(duration) from checkin)"):

query1 = 'select * from checkin'
query2 = 'select DISTINCT(category) from attraction where AttractionID = '
query3 = 'select name from attraction where AttractionID = '
r = 'Rides'
c.execute(query1)
#print(c.fetchall())
checkins = c.fetchall()
r_count = {}
r_dur = {}

for i in checkins:
    if((i[4])):
        c.execute(query2 + str(i[3]))
        category = c.fetchall()
        #print(category[0][0])
        if r in category[0][0]:
            #print(category[0][0])
            r_count.setdefault(i[3],0)
            r_count[i[3]] += 1
            r_dur.setdefault(i[3],0)
            t = getsecs(i[4])
            r_dur[i[3]] += t
            #print(r_dur[i[3]])

av = -1
rid = -1
#print(r_dur)
for i in r_count:
    avt = r_dur[i] / r_count[i]
    if avt > av:
        rid = i
        av = avt
#print(rid)        
c.execute(query3 + str(rid))
result = c.fetchall()
print(result[0][0])

TerrorSaur


In [464]:
# Graded Cell
# PartID: KALua
# Question 3: Which Fast Food offering in the park has the fewest visitors?
# Notes: Your output should be the name of the fast food offering.

ff = pd.read_sql_query('''
Select Name,count(name) from
(select a.attractionID,b.visitorid,a.name FROM checkin as b
INNER JOIN attraction as a
on (b.attraction = a.attractionID
and
a.type = 'Fast Food'
)) GROUP BY Name''',connec)
ff1 = pd.DataFrame(ff) #seq
print(ff1.iloc[-1][0])

Theresaur Food Stop


In [467]:
# Graded Cell
# PartID: B0LUP
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and 
#  report the rides that appear in the Skyline. 
# Notes: Remember that in this case, higher visits is better and lower visit times are better. 
#  Your output should be formatted as an array listing the names of the rides in the Skyline.

query1 = 'select * from checkin'
query2 = 'select DISTINCT(category) from attraction where AttractionID = '
query3 = 'select name from attraction where AttractionID = '
r = 'Rides'
d = []
rides = []
c.execute(query1)
#print(c.fetchall())
checkins = c.fetchall()
r_count = {}
r_dur = {}

for i in checkins:
    if((i[4])):
        c.execute(query2 + str(i[3]))
        category = c.fetchall()
        #print(category[0][0])
        if r in category[0][0]:
            #print(category[0][0])
            r_count.setdefault(i[3],0)
            r_count[i[3]] += 1
            r_dur.setdefault(i[3],0)
            t = getsecs(i[4])
            r_dur[i[3]] += t
        
for i in r_count:
    d.append((r_count[i],r_dur[i]/r_count[i]))
    rides.append((i))
    
dd = pd.DataFrame(d,columns=['Visitors','Tim'])
dr = pd.DataFrame(rides,columns=['r_id'])
#print(dr)
#print(dd)
a = paretoset(dd,sense=['max','min'])
par = dd[a].index.to_list()

res = []

for i in par:
    j = str(dr.iloc[i][0])
    c.execute(query3 + j)
    att = c.fetchall()
    res.append(att[0][0])
    
res

['Keimosaurus Big Spin',
 'Galactosaurus Rage',
 'Wrightiraptor Mountain',
 'Atmosfear',
 'Dykesadactyl Thrill']